In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import torch
from torchvision import transforms

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode
from xdecoder.BaseModel import BaseModel
from xdecoder import build_model
from utils.visualizer import Visualizer
from utils.distributed import init_distributed

import warnings
warnings.filterwarnings(action='ignore')
logger = logging.getLogger(__name__)

from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from datasets import build_evaluator, build_eval_dataloader
from xdecoder.utils import get_class_names

/home/djjin/Mygit/X-Decoder
['xdecoder', 'body', 'decoder', 'xdecoder']


Invalid MIT-MAGIC-COOKIE-1 keyWARNING: Cannot find VLPreDataset. Make sure datasets are accessible if you want to use them for training or evaluation.


datasets/ADE20K_2021_17_01/images_detectron2/training
datasets/ADE20K_2021_17_01/images_detectron2/validation


In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
pretrained_pth = os.path.join(opt['WEIGHT'])
output_root = './output'
image_pth = '../images/animals.png'
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


In [5]:
import matplotlib.pyplot as plt

def plot_image(file_name, figsize=(16,10)):
    raw_image = Image.open(file_name)
    plt.figure(figsize=figsize)
    plt.imshow(raw_image)
    plt.axis('off')
    plt.show()

In [6]:
img_root = home_dir + '/images'
# images = [os.path.join(img_root, f ) for f in os.listdir(img_root)]
# coco_image_paths = []

# for image_pth in images:
#     if os.path.isdir(image_pth):
#         continue
#     if "COCO" in image_pth:
#         coco_image_paths.append(image_pth)
#         plot_image(image_pth, figsize=(4, 4))
coco_image_paths = [os.path.join(img_root, 'COCO_val2014_000000385029.jpg')]

In [7]:
coco_image_paths

['/home/djjin/Mygit/X-Decoder/images/COCO_val2014_000000385029.jpg']

In [8]:
# from typing import Optional, List
# from torch import Tensor
# import datasets.transforms as T

# def _max_by_axis(the_list):
#     # type: (List[List[int]]) -> List[int]
#     maxes = the_list[0]
#     for sublist in the_list[1:]:
#         for index, item in enumerate(sublist):
#             maxes[index] = max(maxes[index], item)
#     return maxes

# class NestedTensor(object):
#     def __init__(self, tensors, mask: Optional[Tensor]):
#         self.tensors = tensors
#         self.mask = mask

#     def to(self, device):
#         cast_tensor = self.tensors.to(device)
#         mask = self.mask
#         if mask is not None:
#             assert mask is not None
#             cast_mask = mask.to(device)
#         else:
#             cast_mask = None
#         return NestedTensor(cast_tensor, cast_mask)

#     def decompose(self):
#         return self.tensors, self.mask

#     def __repr__(self):
#         return str(self.tensors)


# def nested_tensor_from_tensor_list(tensor_list: List[Tensor]):
#     if tensor_list[0].ndim == 3:
#         max_size = _max_by_axis([list(img.shape) for img in tensor_list])
#         batch_shape = [len(tensor_list)] + max_size
#         b, c, h, w = batch_shape
#         dtype = tensor_list[0].dtype
#         device = tensor_list[0].device
#         tensor = torch.zeros(batch_shape, dtype=dtype, device=device)
#         mask = torch.ones((b, h, w), dtype=torch.bool, device=device)
#         for img, pad_img, m in zip(tensor_list, tensor, mask):
#             pad_img[: img.shape[0], : img.shape[1], : img.shape[2]].copy_(img)
#             m[: img.shape[1], : img.shape[2]] = False
#     else:
#         raise ValueError("not supported")
#     return NestedTensor(tensor, mask)

# def load_image(transform, file_path_list, device):
#     raw_image_list = []
#     size_list = []
#     for file_path in file_path_list:
#         raw_image = Image.open(file_path).convert("RGB")
#         img_w, img_h = raw_image.size
#         raw_image_list.append(raw_image)  
#         size_list.append(torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32).to(device))

#     image = [transform(raw_image)[0].to(device) for raw_image in raw_image_list]
#     image = nested_tensor_from_tensor_list(image)
#     size = torch.stack(size_list, dim = 0)
#     return image, raw_image_list, size

# def make_vcoco_transforms(image_set):

#     normalize = T.Compose([
#         T.ToTensor(),
#         T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])

#     scales = [480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800]

#     if image_set == 'train':
#         return T.Compose([
#             T.RandomHorizontalFlip(),
#             T.ColorJitter(.4, .4, .4),
#             T.RandomSelect(
#                 T.RandomResize(scales, max_size=1333),
#                 T.Compose([
#                     T.RandomResize([400, 500, 600]),
#                     T.RandomSizeCrop(384, 600),
#                     T.RandomResize(scales, max_size=1333),
#                 ])
#             ),
#             normalize,
#         ])

#     if image_set == 'val':
#         return T.Compose([
#             T.RandomResize([800], max_size=1333),
#             normalize,
#         ])

#     raise ValueError(f'unknown {image_set}')

# transform = make_vcoco_transforms(image_set = 'val')
# samples, org_images, orig_target_sizes = load_image(transform, coco_image_paths, device='cuda')
# samples.tensors.shape

In [9]:
from copy import deepcopy
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()


*UNLOADED* sem_seg_head.predictor.pos_embed_caping.weight, Model Shape: torch.Size([77, 512])
$UNUSED$ criterion.empty_weight, Ckpt Shape: torch.Size([134])
$UNUSED$ sem_seg_head.predictor.query_feat_caping.weight, Ckpt Shape: torch.Size([77, 512])


In [10]:
from torchvision import transforms
focal_backbone = deepcopy(model.model.backbone)
t = []
t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
transform = transforms.Compose(t)

pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

# focal_backbone.eval()
# evaluate() function in xdecoder_model.py

with torch.no_grad():
    image_ori = Image.open(coco_image_paths[-1]).convert('RGB')
    width = image_ori.size[0]
    height = image_ori.size[1]
    image = transform(image_ori)
    image = np.asarray(image)
    image_ori = np.asarray(image_ori)
    images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
    batch_inputs = [{'image': images, 'height': height, 'width': width}]
    images = [x["image"].to('cuda') for x in batch_inputs]
    images = [(x - pixel_mean) / pixel_std for x in images]
    
    images = ImageList.from_tensors(images, 32)
    img_bs = images.tensor.shape[0]

    print(images.tensor.shape)
    features = focal_backbone(images.tensor)
    for idx, f in enumerate(list(features.keys())[::-1]):
        x = features[f]
        print(x.shape)
        # lateral_conv = self.lateral_convs[idx]
        # output_conv = self.output_convs[idx]

torch.Size([1, 3, 800, 1088])
torch.Size([1, 768, 25, 34])
torch.Size([1, 384, 50, 68])
torch.Size([1, 192, 100, 136])
torch.Size([1, 96, 200, 272])


## Model

In [12]:
t = []
t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
transform = transforms.Compose(t)

thing_classes = ['zebra','antelope','giraffe','ostrich','sky','water','grass','sand','tree']
thing_colors = [random_color(rgb=True, maximum=255).astype(np.int).tolist() for _ in range(len(thing_classes))]
thing_dataset_id_to_contiguous_id = {x:x for x in range(len(thing_classes))}

MetadataCatalog.get("demo").set(
    thing_colors=thing_colors,
    thing_classes=thing_classes,
    thing_dataset_id_to_contiguous_id=thing_dataset_id_to_contiguous_id,
)

model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(thing_classes + ["background"], is_eval=False)
metadata = MetadataCatalog.get('demo')
model.model.metadata = metadata
model.model.sem_seg_head.num_classes = len(thing_classes)

In [13]:
with torch.no_grad():
    image_ori = Image.open(image_pth).convert('RGB')
    width = image_ori.size[0]
    height = image_ori.size[1]
    image = transform(image_ori)
    image = np.asarray(image)
    image_ori = np.asarray(image_ori)
    images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
    batch_inputs = [{'image': images, 'height': height, 'width': width}]
    outputs = model.forward(batch_inputs)

['res5', 'res4', 'res3', 'res2']


In [14]:
# model.model.sem_seg_head.state_dict().keys()

## Backbone

In [15]:
# from copy import deepcopy

# # from xdecoder.backbone.focal import D2FocalNet
# focal_backbone = deepcopy(model.model.backbone)

# # from xdecoder.body.encoder.transformer_encoder_fpn import TransformerEncoderPixelDecoder
# pixel_decoder = deepcopy(model.model.sem_seg_head.pixel_decoder)

# # from xdecoder.body.decoder import TransformerEncoderPixelDecoder
# predictor = model.model.sem_seg_head.predictor


In [16]:
# model.model.sem_seg_head.predictor

In [17]:
# pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
# pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

# # focal_backbone.eval()
# # evaluate() function in xdecoder_model.py
# with torch.no_grad():
#     image_ori = Image.open(image_pth).convert('RGB')
#     width = image_ori.size[0]
#     height = image_ori.size[1]
#     image = transform(image_ori)
#     image = np.asarray(image)
#     image_ori = np.asarray(image_ori)
#     images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
#     batch_inputs = [{'image': images, 'height': height, 'width': width}]
#     images = [x["image"].to('cuda') for x in batch_inputs]
#     images = [(x - pixel_mean) / pixel_std for x in images]
    
#     images = ImageList.from_tensors(images, 32)
#     img_bs = images.tensor.shape[0]

#     # Process
#     # 1. image -> focal_backbone -> features
#     # 2. features -> TransformerEncoderPixelDecoder -> mask_features, transformer_encoder_features, multi_scale_features

#     features = focal_backbone(images.tensor)
#     print(features['res5'].shape)
#     mask_features, transformer_encoder_features, multi_scale_features = pixel_decoder(features)
    
#     print(f"mask_features {mask_features.shape}")
#     print(f"transformer_encoder_features {transformer_encoder_features.shape}")
#     print(f"multi_scale_features {multi_scale_features[0].shape}")

#     # predictor를 수정하면 될듯
#     predictions = predictor(multi_scale_features, mask_features, None, None, None, "seg", {})
#     for key, value in predictions.items():
#         if not isinstance(predictions[key], list):
#             print(key, predictions[key].shape)


In [18]:
# t = []
# t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
# transform = transforms.Compose(t)

# thing_classes = ['zebra','antelope','giraffe','ostrich','sky','water','grass','sand','tree']
# thing_colors = [random_color(rgb=True, maximum=255).astype(np.int).tolist() for _ in range(len(thing_classes))]
# thing_dataset_id_to_contiguous_id = {x:x for x in range(len(thing_classes))}

# MetadataCatalog.get("demo").set(
#     thing_colors=thing_colors,
#     thing_classes=thing_classes,
#     thing_dataset_id_to_contiguous_id=thing_dataset_id_to_contiguous_id,
# )
# model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(thing_classes + ["background"], is_eval=False)
# metadata = MetadataCatalog.get('demo')
# model.model.metadata = metadata
# model.model.sem_seg_head.num_classes = len(thing_classes)

# with torch.no_grad():
#     image_ori = Image.open(image_pth).convert('RGB')
#     width = image_ori.size[0]
#     height = image_ori.size[1]
#     image = transform(image_ori)
#     image = np.asarray(image)
#     image_ori = np.asarray(image_ori)
#     images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()

#     batch_inputs = [{'image': images, 'height': height, 'width': width}]
#     outputs = model.forward(batch_inputs)
#     visual = Visualizer(image_ori, metadata=metadata)

#     inst_seg = outputs[-1]['instances']
#     inst_seg.pred_masks = inst_seg.pred_masks.cpu()
#     inst_seg.pred_boxes = BitMasks(inst_seg.pred_masks > 0).get_bounding_boxes()
#     demo = visual.draw_instance_predictions(inst_seg) # rgb Image

#     if not os.path.exists(output_root):
#         os.makedirs(output_root)
#     demo.save(os.path.join(output_root, 'inst.png'))